# RAG With Vector Search

## Creating Indexed Knowledge Base

In [1]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

## Defining Search Function

In [4]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

## Creating Prompt

In [5]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

## Calling LLM 

In [6]:
from huggingface_hub import InferenceClient

client = InferenceClient()

In [10]:
def llm(prompt):
    response = client.chat_completion(
    model= "meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages = [{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

## Complete RAG Pipeline

In [11]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [12]:
rag("How do I run Kafka?")

'Based on the provided context, I\'ll answer your question.\n\nTo run Kafka, there are a few options based on the context:\n\n1. **Java Kafka:** To run Java-based Kafka components (producer/consumer/kstreams, etc.) in the terminal, you need to run the following command in the project directory:\n\n   ```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\n   Make sure to replace `<jar_name>` with the actual name of your jar file.\n\n2. **Python Kafka:** \n\n   a. To run Python-based Kafka components, you first need to create a virtual environment by running the following commands (run only once):\n\n   ```bash\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\n```\n\n   For Windows, the path is slightly different: `env/Scripts/activate`.\n\n   To activate the virtual environment every time you need it, run:\n\n   ```bash\nsource env/bin/activate\n```\n\n   To deactivate it:\n\n   ```bash\ndeactivate

## Including Vector Search

In [13]:
from qdrant_client import QdrantClient, models

q_client = QdrantClient("http://localhost:6333")

In [15]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

EMBEDDING_DIMENSIONALITY = 512

In [16]:
collection_name = "zoomcamp-faq"

q_client.delete_collection(collection_name=collection_name)

False

In [17]:
# Create the collection with specified vector parameters
q_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [18]:
#Generating points
points = []

for i,doc in enumerate(documents):

    text = doc['question'] + ' ' + doc['text'] 
    vector = models.Document(text=text, model=model_handle)

    point = models.PointStruct(
        id=i,
        vector=models.Document(text=doc['text'], model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        payload=doc
    )
    
    points.append(point)

In [19]:
points[0]

PointStruct(id=0, vector=Document(text="The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register

In [20]:
q_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [57]:
def vector_search(query):

    vector_results = q_client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        query_filter=models.Filter( # filter by course name
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value="data-engineering-zoomcamp")
                )
            ]
        ),
        limit=5, # top closest matches
        with_payload=True #to get metadata in the results
    )

    results = []

    for point in vector_results.points:
        results.append(point.payload)

    return results

In [62]:
vector_search("How do I run Kafka?")

[{'text': 'For example, when running JsonConsumer.java, got:\nConsuming form kafka started\nRESULTS:::0\nRESULTS:::0\nRESULTS:::0\nOr when running JsonProducer.java, got:\nException in thread "main" java.util.concurrent.ExecutionException: org.apache.kafka.common.errors.SaslAuthenticationException: Authentication failed\nSolution:\nMake sure in the scripts in src/main/java/org/example/ that you are running (e.g. JsonConsumer.java, JsonProducer.java), the StreamsConfig.BOOTSTRAP_SERVERS_CONFIG is the correct server url (e.g. europe-west3 from example vs europe-west2)\nMake sure cluster key and secrets are updated in src/main/java/org/example/Secrets.java (KAFKA_CLUSTER_KEY and KAFKA_CLUSTER_SECRET)',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: When running the producer/consumer/etc java scripts, no results retrieved or no message sent',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'If you have this error, it most likely that your kafka broker docker 

In [60]:
def vs_rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [61]:
vs_rag(query="How do I run Kafka?")

"To run Kafka, follow these steps:\n\n1. **Verify the Bootstrap Server URL and Cluster Key/Secrets**:\n   Ensure that in the scripts in `src/main/java/org/example/` (e.g., `JsonConsumer.java`, `JsonProducer.java`), the `StreamsConfig.BOOTSTRAP_SERVERS_CONFIG` is the correct server URL (e.g., `europe-west3` from example vs `europe-west2`). Also, make sure cluster key and secrets are updated in `src/main/java/org/example/Secrets.java` (KAFKA_CLUSTER_KEY and KAFKA_CLUSTER_SECRET).\n\n2. **Start the Kafka Broker Docker Container**:\n   If you are using Docker containers for Kafka, use `docker ps` to confirm that the Kafka broker container is running. If it's not running, navigate to the `docker compose yaml` file folder and run `docker compose up -d` to start all the instances.\n\nThese are the general steps to run Kafka. If you encounter specific issues, refer to the provided solutions in the context."